In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
import lightgbm as lgb

# Load the data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Split the training data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_df['text'], train_df['label'], test_size=0.2, random_state=42)

# Preprocess the text data using CountVectorizer
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(train_data).astype(np.float64)
val_features = vectorizer.transform(val_data).astype(np.float64)
test_features = vectorizer.transform(test_df['text']).astype(np.float64)

# Create a LightGBM classifier and specify its hyperparameters
params = {
    'objective': 'multiclass',
    'num_class': 8,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
clf = lgb.LGBMClassifier(**params)

# Train the LightGBM model
clf.fit(train_features, train_labels, eval_set=[(val_features, val_labels)], early_stopping_rounds=10)

# Evaluate the performance of the model on the validation set
val_pred = clf.predict(val_features)
val_f1_score = f1_score(val_labels, val_pred, average='macro')
print('Validation F1 score:', val_f1_score)

# Make predictions on the test data and save to a CSV file
test_pred = clf.predict(test_features)
submission_df = pd.DataFrame({'id': test_df['id'], 'label': test_pred})
submission_df.to_csv('submission.csv', index=False)

/home/dacon/anaconda3/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.386409 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [W